1 - KÜTÜPHANE & BAŞLATMA

In [1]:
import pandas as pd
import time

print("Veri Temizleme, Feature Engineering işlemi başlıyor...")
start_time = time.time()

Veri Temizleme, Feature Engineering işlemi başlıyor...


2 - VERİ OKUMA

In [2]:
df = pd.read_excel("261700 Toplam Data.xlsx")
print("İlk 5 satır:")
display(df.head())
print(f"Orijinal satır sayısı: {len(df)}")

İlk 5 satır:


,Sıra,Tür,Barkod,Ürün,Tarih,Saat,Miktar,Birim Fiyat
0,1,Satış,8692971095011,İÇİM SÜT YAĞLI 1LT X12,2025-05-18,20:48:36,1.0,45.0
1,2,Satış,8692971095011,İÇİM SÜT YAĞLI 1LT X12,2025-05-12,23:33:52,1.0,45.0
2,3,Satış,8692971095011,İÇİM SÜT YAĞLI 1LT X12,2025-05-11,23:53:54,1.0,45.0
3,4,Satış,8692971095011,İÇİM SÜT YAĞLI 1LT X12,2025-05-11,23:49:26,1.0,45.0
4,5,Satış,8692971095011,İÇİM SÜT YAĞLI 1LT X12,2025-05-10,23:14:40,1.0,45.0


Orijinal satır sayısı: 261700


3 - TEMİZLEME & FİLTRELEME

In [3]:
df = df[df['Tür'] == 'Satış']
df = df[(df['Miktar'] > 0) & (df['Birim Fiyat'] > 0)]

print(f"Filtrelenmiş satır sayısı: {len(df)}")
print("Satış kayıtlarının ilk 5 satırı:")
display(df.head())

Filtrelenmiş satır sayısı: 261566
Satış kayıtlarının ilk 5 satırı:


,Sıra,Tür,Barkod,Ürün,Tarih,Saat,Miktar,Birim Fiyat
0,1,Satış,8692971095011,İÇİM SÜT YAĞLI 1LT X12,2025-05-18,20:48:36,1.0,45.0
1,2,Satış,8692971095011,İÇİM SÜT YAĞLI 1LT X12,2025-05-12,23:33:52,1.0,45.0
2,3,Satış,8692971095011,İÇİM SÜT YAĞLI 1LT X12,2025-05-11,23:53:54,1.0,45.0
3,4,Satış,8692971095011,İÇİM SÜT YAĞLI 1LT X12,2025-05-11,23:49:26,1.0,45.0
4,5,Satış,8692971095011,İÇİM SÜT YAĞLI 1LT X12,2025-05-10,23:14:40,1.0,45.0


4 - DATETIME OLUŞTURMA & KESME

In [4]:
df['Datetime'] = pd.to_datetime(df['Tarih'].astype(str) + ' ' + df['Saat'].astype(str))
df = df[df['Datetime'] < '2025-01-01']
df = df[['Datetime', 'Ürün', 'Miktar', 'Birim Fiyat']]

print("Datetime eklenmiş ve kısıtlanmış veri:")
display(df.head())
print(f"Yeni satır sayısı: {len(df)}")

Datetime eklenmiş ve kısıtlanmış veri:


,Datetime,Ürün,Miktar,Birim Fiyat
63,2024-12-29 20:35:14,İÇİM SÜT YAĞLI 1LT X12,1.0,47.0
64,2024-12-29 20:16:24,İÇİM SÜT YAĞLI 1LT X12,1.0,47.0
65,2024-12-26 23:24:07,İÇİM SÜT YAĞLI 1LT X12,1.0,47.0
66,2024-12-25 19:52:46,İÇİM SÜT YAĞLI 1LT X12,1.0,47.0
67,2024-12-23 22:46:51,İÇİM SÜT YAĞLI 1LT X12,1.0,47.0


Yeni satır sayısı: 235769


5 - ZAMAN ENDİKSLİ YAPILANDIRMA

In [5]:
df = df.set_index('Datetime')

grouped = (
    df.groupby(['Ürün'])
      .resample('5T')
      .agg({'Miktar': 'sum', 'Birim Fiyat': 'mean'})
      .reset_index()
)

print("Gruplandırılmış veri örneği:")
display(grouped.head())

Gruplandırılmış veri örneği:


,Ürün,Datetime,Miktar,Birim Fiyat
0,ABANT SU 500 ML,2022-01-01 15:10:00,1.0,1.25
1,ABANT SU 500 ML,2022-01-01 15:15:00,0.0,NaN
2,ABANT SU 500 ML,2022-01-01 15:20:00,0.0,NaN
3,ABANT SU 500 ML,2022-01-01 15:25:00,0.0,NaN
4,ABANT SU 500 ML,2022-01-01 15:30:00,0.0,NaN


6 - PIVOT TABLOLAR

In [6]:
sales_pivot = grouped.pivot(index='Datetime', columns='Ürün', values='Miktar').fillna(0)
price_pivot = grouped.pivot(index='Datetime', columns='Ürün', values='Birim Fiyat').ffill().bfill()

print("Sales pivot ilk 5 satır:")
display(sales_pivot.head())

print("Price pivot ilk 5 satır:")
display(price_pivot.head())

Sales pivot ilk 5 satır:


Ürün,ABANT SU 500 ML,COCA COLA SISE200ML,CORNETTO KEYIF KORNET,DORİTOS NACHO SÜPER,EKMEK,INDO MIE NOODLE KÖRILI 75 GR.,KARAM GURME 50GX18 KL,KENT D-RANGE BLUE,KINDER PINGUI CACAO T1 - 30,LAY S YOĞURT MEVSİM YEŞİLLİKLERİ SÜPER,LÜKS KARIŞIK,NESCAFE 3Ü1 ARADA SÜTLÜ KÖPÜKLÜ CLEO 12(48X17.4G),SIYAH DAKOTA ÇEKİRDEK,TURK KAHVESI,ÜLKER LAVİVA DOLG.VE BİSK.ÇİK.35GRX24X6,İÇİM SÜT YAĞLI 1LT X12
Datetime,,,,,,,,,,,,,,,,
2022-01-01 00:05:00,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.13,0.0,0.0,0.0
2022-01-01 00:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.25,0.0,0.0,0.0
2022-01-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.49,0.0,0.0,0.0
2022-01-01 00:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.15,0.0,0.00,0.0,2.0,0.0
2022-01-01 00:25:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0


Price pivot ilk 5 satır:


Ürün,ABANT SU 500 ML,COCA COLA SISE200ML,CORNETTO KEYIF KORNET,DORİTOS NACHO SÜPER,EKMEK,INDO MIE NOODLE KÖRILI 75 GR.,KARAM GURME 50GX18 KL,KENT D-RANGE BLUE,KINDER PINGUI CACAO T1 - 30,LAY S YOĞURT MEVSİM YEŞİLLİKLERİ SÜPER,LÜKS KARIŞIK,NESCAFE 3Ü1 ARADA SÜTLÜ KÖPÜKLÜ CLEO 12(48X17.4G),SIYAH DAKOTA ÇEKİRDEK,TURK KAHVESI,ÜLKER LAVİVA DOLG.VE BİSK.ÇİK.35GRX24X6,İÇİM SÜT YAĞLI 1LT X12
Datetime,,,,,,,,,,,,,,,,
2022-01-01 00:05:00,1.25,4.0,4.0,8.0,2.25,3.5,3.5,18.0,2.75,8.0,139.0,1.25,40.0,180.0,3.0,11.5
2022-01-01 00:10:00,1.25,4.0,4.0,8.0,2.25,3.5,3.5,18.0,2.75,8.0,139.0,1.25,40.0,180.0,3.0,11.5
2022-01-01 00:15:00,1.25,4.0,4.0,8.0,2.25,3.5,3.5,18.0,2.75,8.0,139.0,1.25,40.0,180.0,3.0,11.5
2022-01-01 00:20:00,1.25,4.0,4.0,8.0,2.25,3.5,3.5,18.0,2.75,8.0,139.0,1.25,40.0,180.0,3.0,11.5
2022-01-01 00:25:00,1.25,4.0,4.0,8.0,2.25,3.5,3.5,18.0,2.75,8.0,139.0,1.25,40.0,180.0,3.0,11.5


7 - KOLON İSİMLERİ & BİRLEŞTİRME

In [7]:
sales_pivot.columns = [f"{col}_miktar" for col in sales_pivot.columns]
price_pivot.columns = [f"{col}_fiyat" for col in price_pivot.columns]

pivot_final = sales_pivot.join(price_pivot).sort_index()
print("Pivot final ilk 5 satır:")
display(pivot_final.head())

Pivot final ilk 5 satır:


,ABANT SU 500 ML_miktar,COCA COLA SISE200ML_miktar,CORNETTO KEYIF KORNET_miktar,DORİTOS NACHO SÜPER_miktar,EKMEK_miktar,INDO MIE NOODLE KÖRILI 75 GR._miktar,KARAM GURME 50GX18 KL_miktar,KENT D-RANGE BLUE_miktar,KINDER PINGUI CACAO T1 - 30_miktar,LAY S YOĞURT MEVSİM YEŞİLLİKLERİ SÜPER_miktar,...,KARAM GURME 50GX18 KL_fiyat,KENT D-RANGE BLUE_fiyat,KINDER PINGUI CACAO T1 - 30_fiyat,LAY S YOĞURT MEVSİM YEŞİLLİKLERİ SÜPER_fiyat,LÜKS KARIŞIK_fiyat,NESCAFE 3Ü1 ARADA SÜTLÜ KÖPÜKLÜ CLEO 12(48X17.4G)_fiyat,SIYAH DAKOTA ÇEKİRDEK_fiyat,TURK KAHVESI_fiyat,ÜLKER LAVİVA DOLG.VE BİSK.ÇİK.35GRX24X6_fiyat,İÇİM SÜT YAĞLI 1LT X12_fiyat
Datetime,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:05:00,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,3.5,18.0,2.75,8.0,139.0,1.25,40.0,180.0,3.0,11.5
2022-01-01 00:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.5,18.0,2.75,8.0,139.0,1.25,40.0,180.0,3.0,11.5
2022-01-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.5,18.0,2.75,8.0,139.0,1.25,40.0,180.0,3.0,11.5
2022-01-01 00:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.5,18.0,2.75,8.0,139.0,1.25,40.0,180.0,3.0,11.5
2022-01-01 00:25:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.5,18.0,2.75,8.0,139.0,1.25,40.0,180.0,3.0,11.5


8 - FEATURE ENGINEERING

In [8]:
df_fe = pivot_final.copy()

df_fe['hour'] = df_fe.index.hour
df_fe['day'] = df_fe.index.day
df_fe['weekday'] = df_fe.index.weekday
df_fe['is_weekend'] = df_fe['weekday'].isin([5, 6]).astype(int)

miktar_cols = [col for col in df_fe.columns if col.endswith('_miktar')]

for col in miktar_cols:
    lag_col = f"{col}_lag1"
    roll_col = f"{col}_roll5"
    df_fe[lag_col] = df_fe[col].shift(1)
    df_fe[roll_col] = df_fe[col].shift(1).rolling(window=5).mean()

df_fe.fillna(0, inplace=True)

print("Feature engineered veri örneği:")
display(df_fe.head())

Feature engineered veri örneği:


,ABANT SU 500 ML_miktar,COCA COLA SISE200ML_miktar,CORNETTO KEYIF KORNET_miktar,DORİTOS NACHO SÜPER_miktar,EKMEK_miktar,INDO MIE NOODLE KÖRILI 75 GR._miktar,KARAM GURME 50GX18 KL_miktar,KENT D-RANGE BLUE_miktar,KINDER PINGUI CACAO T1 - 30_miktar,LAY S YOĞURT MEVSİM YEŞİLLİKLERİ SÜPER_miktar,...,NESCAFE 3Ü1 ARADA SÜTLÜ KÖPÜKLÜ CLEO 12(48X17.4G)_miktar_lag1,NESCAFE 3Ü1 ARADA SÜTLÜ KÖPÜKLÜ CLEO 12(48X17.4G)_miktar_roll5,SIYAH DAKOTA ÇEKİRDEK_miktar_lag1,SIYAH DAKOTA ÇEKİRDEK_miktar_roll5,TURK KAHVESI_miktar_lag1,TURK KAHVESI_miktar_roll5,ÜLKER LAVİVA DOLG.VE BİSK.ÇİK.35GRX24X6_miktar_lag1,ÜLKER LAVİVA DOLG.VE BİSK.ÇİK.35GRX24X6_miktar_roll5,İÇİM SÜT YAĞLI 1LT X12_miktar_lag1,İÇİM SÜT YAĞLI 1LT X12_miktar_roll5
Datetime,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:05:00,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-01 00:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.13,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-01 00:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.49,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-01 00:25:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,2.0,0.0,0.0,0.0


9 - KAYDETME

In [9]:
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Son veri seti boyutu: {df_fe.shape}")
df_fe.to_excel("pivot_final_with_features_ALL.xlsx")

print(f"Toplam süre: {elapsed_time:.2f} saniye")
print("İşlem tamamlandı ve dosya kaydedildi.")

Son veri seti boyutu: (315645, 68)
Toplam süre: 54.19 saniye
İşlem tamamlandı ve dosya kaydedildi.
